In [1]:
# %pip install nilearn
# %pip install openpyxl
# %pip install Path
# %pip install seaborn
# %pip install nltools
# %pip install networkx

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import scipy.io
import networkx as nx

In [2]:
icn = pd.read_csv('ADNI_demos.txt')
#icn.info()

for i in range(2404):
  icn.at[i,"fc_dir"] = icn.iloc[i]["fc_dir"].replace("FC","GIGICA")    # Replacing FC with GIGICA

ADs = list()
CNs = list()

for i in range(2404):
    if icn["ResearchGroup"][i] == "AD":
        ADs.append(icn["fc_dir"][i])
    elif icn["ResearchGroup"][i] == "CN":
        CNs.append(icn["fc_dir"][i])

In [25]:
component_matFile = "adni_aa__postprocess_results.mat"
component_key = "fnc_corrs_all"

In [26]:
componentIndexes = pd.read_csv('NM_icns_info.csv')
#componentIndexes.info()
domainList = componentIndexes["icn_domain"].tolist()
indexList = componentIndexes["icn_index"].tolist()

In [27]:
def createDataObjects(filePaths):
    ans = dict()
    for i in range(len(filePaths)):
        imageName = "image"+str(i+1)
        ans[imageName]=dict()
        ans[imageName]["fnc_matrix"]=np.zeros((53,53), dtype=np.float64).reshape(53,53)
        ans[imageName]["graphMetrics"] = dict()
        ans[imageName]["graphMetrics"]["betweenessCentrality"] = 0
        ans[imageName]["graphMetrics"]["clusterCoefficient"] = 0
    return ans

AD_FNC = createDataObjects(ADs)
CN_FNC = createDataObjects(CNs)

In [28]:
def loadMatFile(filePath,key):
    componentDict = scipy.io.loadmat(filePath+component_matFile)
    return componentDict[key]

def prepareFNCGraph(components):
    
    selected_component = np.zeros((53,53), dtype=np.float64).reshape(53,53)
    for i in range(53):
        for j in range(i+1, 53):
            # if components[indexList[i]-1][indexList[j]-1] >=0 : 
            selected_component[i][j]=components[indexList[i]-1][indexList[j]-1]
    selected_component += selected_component.T
    return selected_component

def loadFNCGraphForEachImage(subjectFNC, filePaths):
    for i in range(len(filePaths)):
        imageName = "image"+str(i+1)
        components = loadMatFile(filePaths[i],component_key)
        subjectFNC[imageName]["fnc_matrix"] = prepareFNCGraph(components)
        if i==0:
            break
        # prepareGraphMetrics(subjectFNC[imageName])

def prepareHeatmap(component):
    corrs = pd.DataFrame(component)
    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(
            corrs.corr(), cmap='coolwarm', square=True, ax=ax, linewidth='0.5', center=0,
            xticklabels=domainList, yticklabels=domainList,  annot=False)


def prepareGraphMetrics(inputFNCMatrix):
    current_graph = nx.from_numpy_array(inputFNCMatrix["fnc_matrix"])
    # inputFNCMatrix["graphMetrics"]["clusterCoefficient"]   =  nx.cluster.average_clustering(current_graph)
    betweenees = nx.algorithms.approximate_current_flow_betweenness_centrality(current_graph)
    val = []
    for i in range(53):
        val.append(betweenees[i])
    inputFNCMatrix["graphMetrics"]["betweenessCentrality"] =  np.average(val)

In [50]:
loadFNCGraphForEachImage(AD_FNC,ADs)
# loadFNCGraphForEachImage(CN_FNC,CNs)

In [103]:
# prepareHeatmap(AD_FNC["image1"]["fnc_matrix"])
# AD_FNC["image1"]["fnc_matrix"]

corrs = pd.DataFrame(AD_FNC["image1"]["fnc_matrix"])
correlation_matrix = corrs.corr()

correlation_numpy_array = correlation_matrix.to_numpy(dtype=np.float64)
row,column = correlation_numpy_array.shape

for i in range(row):
        for j in range(column):
                if i!=j and correlation_numpy_array[i][j]<0:
                        correlation_numpy_array[i][j]=0

input_matrix = nx.from_numpy_array(correlation_numpy_array)
d = nx.modularity_matrix(input_matrix, weight='weight')
# print(row,column)
# print(d[0].tolist()[1][3])
a=list()
for i in range(row):
        s=0
        k=d[i].tolist()[0]
        for j in range(0,column):
                s+=k[j]
        # print(s)
        a.append(s*s)

e=list()
for i in range(d.shape[0]):
        xl=d[i].tolist()[0][i]
        e.append(xl)
# print(np.sum(e),np.max(e))
# print(np.sum(a), np.min(a))

ans=0
for i in range(row):
        ans+=(e[i]-a[i])
print(ans)
# modularity = e-a
# print(modularity)
# input_matrix = np.matrix(corrs.corr(), dtype=np.float64)

# print(corrs)
# # input_graph = nx.from_numpy_matrix(input_matrix)
# # prepareHeatmap(AD_FNC["image1"]["fnc_matrix"])
# fig, ax = plt.subplots(figsize=(10,10))
# sns.heatmap(

        
#         correlation_numpy_array, cmap='coolwarm', square=True, ax=ax, linewidth='0.5', center=0,
#         xticklabels=domainList, yticklabels=domainList,  annot=False)

# # sns.heatmap(
# #         corrs.corr(), cmap='coolwarm', square=True, ax=ax, linewidth='0.5', center=0,
# #         xticklabels=domainList, yticklabels=domainList,  annot=False)
# print(nx.global_efficiency(nx.from_numpy_matrix(AD_FNC["image1"]["fnc_matrix"])))

43.44941549341944
